In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#import pandas as pd
chord_base = ['C#m', 'C#', 'Cm', 'C', 'D#m', 'D#', 'Dm', 'D', 'Em', 'E', 'F#m', 'F#', 'Fm', 'F', 'G#m', 'G#', 'Gm', 'G', 'A#m', 'A#', 'Am', 'A', 'Bm', 'B']
class_pitch = {0:'C', 1:'C#', 2:'D', 3:'D#', 4:'E', 5:'F', 6:'F#', 7:'G', 8:'G#', 9:'A', 10:'A#', 11:'B'}
chord_map = pd.read_table('ChordDistanceMap.csv', sep=",", index_col="Chords")

NameError: name 'pd' is not defined

# Create Normalized Chords

In [ ]:
def sepExt(chord):
    
    #if '/' in chord:
    #    chord_type_split = chord.split('/')
    #else:
    #chord_type_split = [chord_type]
    #for chord in chord_type_split:
    
    #Em -> Em
    #Emaj7 -> E
    #Emmaj -> Em
    
    for note in chord_base:
        #print('{}, {}'.format(note, chord))
        if note in chord:
            
            if chord.replace(note, '')[:2] == 'aj':
                continue
            else:
                ext = chord.replace(note, '')
                #if len(ext) > 0:
                    #print('{}, {}, {}, {}'.format(chord, note, chord[len(note):len(note)+2], ext))
                return note, ext

In [ ]:
def buildNormChord(center, chord):

    #Dm - maj7
    #Center:
    #full:2X0Xmaj7
    #partial: 2X0
    #bare: 2
    chord_stem, ext = sepExt(chord)
    chord_root = chord_stem.replace('m','')
    distance = chord_map[center][chord_root]
    if chord_stem[len(chord_stem)-1] == 'm':
        isMajor = '0'
    else:
        isMajor = '1'
    
    
    norm_chord_bare = str(distance)
    norm_chord_partial = norm_chord_bare + 'X{}'.format(isMajor)
    if len(ext) > 0:
        norm_chord_full = norm_chord_partial + 'X{}'.format(ext)
    else:
        norm_chord_full = norm_chord_partial
    
    return norm_chord_full, norm_chord_partial, norm_chord_bare

In [ ]:
def vectorizeTab(chords, ngram_low, ngram_high,  binary, Tfidf = False, runSVD = 0, num_components = 100, train_perc = 0.8, min_df = 100):
    if Tfidf:
        vectorizer = TfidfVectorizer(max_features=100000, ngram_range=(ngram_low,ngram_high), lowercase=True, strip_accents=None, binary=binary)
    else:
        vectorizer = CountVectorizer(max_features=100000, ngram_range=(ngram_low,ngram_high), lowercase=True, strip_accents=None, binary=binary)
    
    train_break = int(len(df)*train_perc)
    
    X_train_ChordProg =  vectorizer.fit_transform(chords[:train_break])
    X_test_ChordProg = vectorizer.transform(chords[train_break:])
    
    if runSVD == 1:
        try:
            svd = TruncatedSVD(int(num_components))
            X_train_ChordProg = svd.fit_transform(X_train_ChordProg)
            X_test_ChordProg = svd.transform(X_test_ChordProg)
            #print(X_train_ChordProg.shape)
            #print(X_test_ChordProg.shape)
        except Exception as e:
            print('Didnt Run SVD')
            print(e)
            pass

    
    vocab = dict(sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1]))
    
    return vocab, X_train_ChordProg, X_test_ChordProg

In [ ]:
def buildNormTab(df, center_col):
    
    #tab_full_dict = {}
    #tab_partial_dict = {}
    #tab_bare_dict = {}
    
    full = []
    partial = []
    bare = []
    for idx in df.index:
        center = class_pitch[df.loc[idx, center_col]]
        song_full = []
        song_partial = []
        song_bare = []
        for chord in df.loc[idx, 'clean_tabs'].split(','):
            if '/' in chord:
                chords = chord.split('/')
                chord_full_one, chord_partial_one, chord_bare_one = buildNormChord(center, chords[0])
                chord_full_two, chord_partial_two, chord_bare_two = buildNormChord(center, chords[1])
                chord_full = '{}/{}'.format(chord_full_one, chord_full_two)
                chord_partial = '{}/{}'.format(chord_partial_one, chord_partial_two)
                chord_bare = '{}/{}'.format(chord_bare_one, chord_bare_two)
            else:
                chord_full, chord_partial, chord_bare = buildNormChord(center, chord)

            song_full.append(chord_full)
            song_partial.append(chord_partial)
            song_bare.append(chord_bare)

        full.append(' '.join(song_full))
        partial.append(' '.join(song_partial))
        bare.append(' '.join(song_bare))
        
    return full, partial, bare